<a href="https://colab.research.google.com/github/vggls/medical_xai/blob/main/experiments/non_medical_datasets_haas/CIFAR_10_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports**

In [ ]:
pip install grad-cam

In [ ]:
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
from pytorch_grad_cam import HiResCAM
import pickle
import os
from torch.utils.data import DataLoader

In [ ]:
from training_loop import Train
from train_model import fit
from haas import Haas, HA_image
from testing_report import Test_Report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Load Data**

In [ ]:
transforms = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_and_val_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
classes = train_and_val_set.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [ ]:
len(train_and_val_set), len(testset)

(50000, 10000)

In [ ]:
val_size = 10000
train_size = len(train_and_val_set) - val_size
trainset, validationset = random_split(train_and_val_set, [train_size, val_size])
len(trainset), len(validationset)

(40000, 10000)

## **Custom VGG19 model**

In [ ]:
import collections
from torchvision import models
import torch.nn as nn

class VGG19_CIFAR10():

    def __init__(self,
                 num_classes,
                 trainable_feature_layers=None):

        self.num_classes = num_classes
        self.trainable_feature_layers = trainable_feature_layers

        vgg19 = models.vgg19(pretrained=True)

        features = vgg19.features[:17] # as per paper HAAS paper FIGURE 4 (b)

        vgg19.avgpool = nn.Flatten()
        flatten = vgg19.avgpool

        classifier = nn.Linear(256 * 8 * 8, self.num_classes)

        self.model = nn.Sequential(collections.OrderedDict([
            ("features", features),
            ("flatten", flatten),
            ("classifier", classifier)]
        ))

        # LAYERS TO FREEZE DURING TRAINING
        if self.trainable_feature_layers==None:
            self.freeze = self.model.features
        else:
            assert all(x in range(len(self.model.features)) for x in self.trainable_feature_layers)
            self.freeze = [self.model.features[j] for j in range(len(self.model.features)) \
                           if j not in self.trainable_feature_layers]

## **Loop (16 models)**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/CIFAR-10/'

count = 0

accuracies, macro_f1_scores, mean_roc_auc_scores = [], [], []
haas_scores = []

for lr in [0.0001, 0.000001]:
    for wd in ['yes', 'no']:
        for bs in [32, 64, 128, 256]:

            count += 1

            vgg19 = VGG19_CIFAR10(num_classes=10,
                          trainable_feature_layers=[i for i in range(17)])

            loss_fct = torch.nn.CrossEntropyLoss()

            if wd == 'yes':
              optimizer = optim.Adam(vgg19.model.parameters(), lr=lr, weight_decay=0.01)
            else:
              optimizer = optim.Adam(vgg19.model.parameters(), lr=lr)

            print('\033[1m' + 'Model no.{} - lr:{} - wd:{} - bs:{}'.format(count, lr, wd, bs) + '\033[0m')

            training_dict, validation_dict = fit(train_dataset=trainset,
                                                validation_dataset=validationset,
                                                batch_size=bs,
                                                model=vgg19,
                                                loss_fct=loss_fct,
                                                optimizer=optimizer,
                                                scheduler=None,
                                                epochs=10,
                                                patience=None,
                                                no_of_classes=10,
                                                labels_of_normal_classes=None)

            model = torch.load('model_epoch10.pt')
            model = model.cuda()
            model = model.eval()

            # MODEL CLASSIFICATION METRICS
            test_loader = DataLoader(dataset=testset, batch_size=128, shuffle=False)
            test_report = Test_Report(test_loader, model, classes)

            accuracies.append(test_report.accuracy())

            macro_f1_scores.append(test_report.f1('macro'))

            model_roc_auc_scores = test_report.roc_curve_and_scores(False)
            mean_roc_auc_scores.append(sum(model_roc_auc_scores)/len(model_roc_auc_scores))

            # MODEL HAAS SCORE
            print('HiResCAM - HAAS calculation')
            hirescam_instance = HiResCAM(model=model, target_layers=[model.features[-1]], use_cuda=True)
            hirescam_score, _, _, _ = Haas(testset, model, hirescam_instance)
            haas_scores.append(hirescam_score)
            print('HAAS score:', hirescam_score)

            if hirescam_score>1:
              torch.save(model, drive_path + 'cifar10_vgg19_no{}.pt'.format(count))
              with open(drive_path + 'cifar10_vgg19_no{}_training_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(training_dict, f)
              with open(drive_path + 'cifar10_vgg19_no{}_validation_dict.pickle'.format(count), 'wb') as f:
                pickle.dump(validation_dict, f)

            os.remove('model_epoch10.pt')

            print('*'*100)
            print('\n')

with open(drive_path + 'cifar10_haas_scores.pickle', 'wb') as f: pickle.dump(haas_scores, f)
with open(drive_path + 'cifar10_accuracies.pickle', 'wb') as f: pickle.dump(accuracies, f)
with open(drive_path + 'cifar10_macro_f1_scores.pickle', 'wb') as f: pickle.dump(macro_f1_scores, f)
with open(drive_path + 'cifar10_mean_roc_auc_scores.pickle', 'wb') as f: pickle.dump(mean_roc_auc_scores, f)

Model no.1 - lr:0.0001 - wd:yes - bs:32
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=0.7725
  Accuracy=0.75
->Validation phase
  Loss=0.5481
  Accuracy=0.8
-----------------------------------
Epoch 2/10
->Training phase
  Loss=0.4771
  Accuracy=0.83
->Validation phase
  Loss=0.4933
  Accuracy=0.83
-----------------------------------
Epoch 3/10
->Training phase
  Loss=0.4598
  Accuracy=0.84
->Validation phase
  Loss=0.5610
  Accuracy=0.8
-----------------------------------
Epoch 4/10
->Training phase
  Loss=0.4565
  Accuracy=0.84
->Validation phase
  Loss=0.5370
  Accuracy=0.82
-----------------------------------
Epoch 5/10
->Training phase
  Loss=0.4607
  Accuracy=0.84
->Validation phase
  Loss=0.5149
  Accuracy=0.82
-----------------------------------
Epoch 6/10
->Training phase
  Loss=0.4626
  Accuracy=0.84
->Validation phase
  Loss=0.5237
  Accuracy=0.82
-----------------------------------
Epoch 7/10
->Training phase
  Los

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.2 - lr:0.0001 - wd:yes - bs:64
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=0.8496
  Accuracy=0.73
->Validation phase
  Loss=0.5612
  Accuracy=0.81
-----------------------------------
Epoch 2/10
->Training phase
  Loss=0.4707
  Accuracy=0.84
->Validation phase
  Loss=0.5351
  Accuracy=0.81
-----------------------------------
Epoch 3/10
->Training phase
  Loss=0.4054
  Accuracy=0.86
->Validation phase
  Loss=0.4780
  Accuracy=0.84
-----------------------------------
Epoch 4/10
->Training phase
  Loss=0.3827
  Accuracy=0.87
->Validation phase
  Loss=0.5095
  Accuracy=0.82
-----------------------------------
Epoch 5/10
->Training phase
  Loss=0.3784
  Accuracy=0.87
->Validation phase
  Loss=0.5712
  Accuracy=0.8
-----------------------------------
Epoch 6/10
->Training phase
  Loss=0.3819
  Accuracy=0.87
->Validation phase
  Loss=0.4973
  Accuracy=0.83
-----------------------------------
Epoch 7/10
->Training phase
  Lo

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.3 - lr:0.0001 - wd:yes - bs:128
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=0.9518
  Accuracy=0.71
->Validation phase
  Loss=0.5957
  Accuracy=0.8
-----------------------------------
Epoch 2/10
->Training phase
  Loss=0.4996
  Accuracy=0.83
->Validation phase
  Loss=0.5072
  Accuracy=0.83
-----------------------------------
Epoch 3/10
->Training phase
  Loss=0.3954
  Accuracy=0.86
->Validation phase
  Loss=0.4523
  Accuracy=0.84
-----------------------------------
Epoch 4/10
->Training phase
  Loss=0.3446
  Accuracy=0.88
->Validation phase
  Loss=0.4579
  Accuracy=0.84
-----------------------------------
Epoch 5/10
->Training phase
  Loss=0.3295
  Accuracy=0.89
->Validation phase
  Loss=0.4550
  Accuracy=0.84
-----------------------------------
Epoch 6/10
->Training phase
  Loss=0.3076
  Accuracy=0.9
->Validation phase
  Loss=0.4518
  Accuracy=0.84
-----------------------------------
Epoch 7/10
->Training phase
  Lo

...

Model no.14 - lr:1e-06 - wd:no - bs:64
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=4.8567
  Accuracy=0.21
->Validation phase
  Loss=2.7877
  Accuracy=0.34
-----------------------------------
Epoch 2/10
->Training phase
  Loss=2.1624
  Accuracy=0.43
->Validation phase
  Loss=1.8139
  Accuracy=0.49
-----------------------------------
Epoch 3/10
->Training phase
  Loss=1.5550
  Accuracy=0.54
->Validation phase
  Loss=1.4331
  Accuracy=0.56
-----------------------------------
Epoch 4/10
->Training phase
  Loss=1.2770
  Accuracy=0.6
->Validation phase
  Loss=1.2283
  Accuracy=0.61
-----------------------------------
Epoch 5/10
->Training phase
  Loss=1.1137
  Accuracy=0.64
->Validation phase
  Loss=1.1041
  Accuracy=0.64
-----------------------------------
Epoch 6/10
->Training phase
  Loss=1.0038
  Accuracy=0.67
->Validation phase
  Loss=1.0135
  Accuracy=0.67
-----------------------------------
Epoch 7/10
->Training phase
  Los

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.15 - lr:1e-06 - wd:no - bs:128
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=5.3866
  Accuracy=0.17
->Validation phase
  Loss=3.6449
  Accuracy=0.25
-----------------------------------
Epoch 2/10
->Training phase
  Loss=2.9201
  Accuracy=0.32
->Validation phase
  Loss=2.4028
  Accuracy=0.39
-----------------------------------
Epoch 3/10
->Training phase
  Loss=2.0926
  Accuracy=0.43
->Validation phase
  Loss=1.8620
  Accuracy=0.47
-----------------------------------
Epoch 4/10
->Training phase
  Loss=1.6876
  Accuracy=0.5
->Validation phase
  Loss=1.5663
  Accuracy=0.53
-----------------------------------
Epoch 5/10
->Training phase
  Loss=1.4492
  Accuracy=0.55
->Validation phase
  Loss=1.3820
  Accuracy=0.57
-----------------------------------
Epoch 6/10
->Training phase
  Loss=1.2920
  Accuracy=0.59
->Validation phase
  Loss=1.2517
  Accuracy=0.6
-----------------------------------
Epoch 7/10
->Training phase
  Los

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.16 - lr:1e-06 - wd:no - bs:256
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=5.8544
  Accuracy=0.15
->Validation phase
  Loss=4.5316
  Accuracy=0.21
-----------------------------------
Epoch 2/10
->Training phase
  Loss=3.8246
  Accuracy=0.25
->Validation phase
  Loss=3.2786
  Accuracy=0.3
-----------------------------------
Epoch 3/10
->Training phase
  Loss=2.8681
  Accuracy=0.34
->Validation phase
  Loss=2.5395
  Accuracy=0.38
-----------------------------------
Epoch 4/10
->Training phase
  Loss=2.3090
  Accuracy=0.41
->Validation phase
  Loss=2.1300
  Accuracy=0.43
-----------------------------------
Epoch 5/10
->Training phase
  Loss=1.9537
  Accuracy=0.46
->Validation phase
  Loss=1.8534
  Accuracy=0.48
-----------------------------------
Epoch 6/10
->Training phase
  Loss=1.7154
  Accuracy=0.51
->Validation phase
  Loss=1.6829
  Accuracy=0.51
-----------------------------------
Epoch 7/10
->Training phase
  Lo

## **Summary**

In [ ]:
import pickle

drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/CIFAR-10/'

with open(drive_path + "cifar10_mean_roc_auc_scores.pickle", "rb") as f: auc_scores = pickle.load(f)
with open(drive_path + "cifar10_macro_f1_scores.pickle", "rb") as f: f1_scores = pickle.load(f)
with open(drive_path + "cifar10_haas_scores.pickle", "rb") as f: haas_scores = pickle.load(f)
with open(drive_path + "cifar10_accuracies.pickle", "rb") as f: accuracies = pickle.load(f)

In [ ]:
import pandas as pd

n = [i for i in range(1,17)]
models = ['vgg19_no{}'.format(i) for i in n]

dictionary = {'Model':models,
              'HAAS score':haas_scores,
              'Mean AUC score':[round(score,3) for score in auc_scores],
              'Accuracy':[round(score,3) for score in accuracies],
              'Macro F1':[round(score,3) for score in f1_scores]}

df = pd.DataFrame.from_dict(dictionary)
df.style.set_properties(**{'text-align': 'center'})
df

,Model,HAAS score,Mean AUC score,Accuracy,Macro F1
0,vgg19_no1,1.009,0.981,0.795,0.792
1,vgg19_no2,1.007,0.985,0.826,0.827
2,vgg19_no3,0.995,0.985,0.830,0.829
3,vgg19_no4,1.004,0.987,0.835,0.836
4,vgg19_no5,0.981,0.987,0.844,0.843
5,vgg19_no6,0.987,0.987,0.845,0.845
6,vgg19_no7,0.986,0.987,0.852,0.852
7,vgg19_no8,0.996,0.987,0.846,0.846
8,vgg19_no9,0.983,0.969,0.758,0.757
9,vgg19_no10,0.981,0.959,0.729,0.729
